In [1]:
import pandas as pd
import numpy as np
import sklearn
import transformers
import torch
import torch.nn as nn
import torch.functional as f

from transformers import DistilBertTokenizer, DistilBertModel

from sklearn.metrics.pairwise import cosine_similarity

model = DistilBertModel.from_pretrained('bert-base-multilingual-cased')

tokenizer = DistilBertTokenizer.from_pretrained('bert-base-multilingual-cased')

import copy

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['transformer.layer.3.ffn.lin2.bias', 'transformer.layer.4.ffn.lin2.weight', 'transformer.layer.2.attention.out_lin.bias', 'transformer.layer.5.sa_layer_norm.bias', 'transformer.layer.8.attention.v_lin.weight', 'transformer.layer.9.ffn.lin2.weight', 'transformer.layer.4.attention.v_lin.bias', 'transformer.layer.9.sa_layer_norm.bias', 'transformer.layer.4.ffn.lin1.bias', 'transformer.layer.5.attention.k_lin.bias', 'transformer.layer.2.ffn.lin1.bias', 'transformer.layer.7.sa_layer_norm.weight', 'embeddings.position_embeddings.weight', 'transformer.layer.7.attention.k_lin.weight', 'transformer.layer.9.ffn.lin2.bias', 'transformer.layer.10.ffn.lin1.bias', 'transformer.layer.7.attention.v_lin.weight', '

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


In [2]:
df = pd.read_csv("./df_output4.csv")

In [3]:
model.to('cuda')

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-11): 12 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2):

In [4]:
dict_desc = {}
for i in range(len(df)):
    try:
        text = df.loc[i, 'description']
        input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors='pt').to('cuda')
        outputs = model(input_ids)
        
        embedding = outputs.last_hidden_state.to('cpu')
        
        embed_mtx = embedding.reshape(embedding.shape[0], -1).detach().numpy()
        
        dict_desc[i] = embed_mtx
    except:
        continue
        
    print(i)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
cmd = pd.DataFrame()

In [22]:
for i in range(len(dict_desc)):
    for j in range(i, len(dict_desc)):
        cmd.loc[i, j] = float(cosine_similarity(dict_desc[i], dict_desc[j])[0])
        
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [40]:
trs = copy.deepcopy(df['translations'])

In [42]:
trs = trs.fillna('')

In [46]:
dict_trs = {}
for i in range(len(df)):
    try:
        text = trs[i]
        input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors='pt').to('cuda')
        outputs = model(input_ids)
        
        embedding = outputs.last_hidden_state.to('cpu')
        
        embed_mtx = embedding.reshape(embedding.shape[0], -1).detach().numpy()
        
        dict_trs[i] = embed_mtx
    except:
        continue
        
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [32]:
dict_trs

{0: array([[-1.0160213 , -0.703496  , -0.87391555, ..., -0.67801434,
          0.684012  , -0.34797305]], dtype=float32),
 1: array([[-1.0043027 , -0.59865904, -0.79504144, ..., -0.58390915,
          0.6265842 , -0.5850644 ]], dtype=float32),
 2: array([[-1.0480046 , -0.641331  , -0.6809121 , ..., -0.6613338 ,
          0.516308  , -0.55109423]], dtype=float32),
 3: array([[-1.0768785 , -0.62644637, -0.8805338 , ..., -0.6270703 ,
          0.556864  , -0.5262853 ]], dtype=float32),
 4: array([[-1.0076469 , -0.64612967, -0.765153  , ..., -0.52877116,
          0.60847276, -0.44590405]], dtype=float32),
 5: array([[-1.077859  , -0.6246345 , -0.8788513 , ..., -0.59617543,
          0.79256237, -0.43628642]], dtype=float32),
 6: array([[-1.005785  , -0.80392295, -0.8454047 , ..., -0.59993476,
          0.6213421 , -0.4365875 ]], dtype=float32),
 7: array([[-1.0675313 , -0.6259424 , -0.828841  , ..., -0.5736251 ,
          0.5056066 , -0.58144766]], dtype=float32),
 8: array([[-0.98732764,

In [10]:
import warnings
warnings.filterwarningsarnings('ignore')

# translations 유사도 계산

In [47]:
cos_trans = pd.DataFrame()
for i in range(len(dict_trs)):
    for j in range(i, len(dict_trs)):
        try: 
            cos_trans.loc[i, j] = float(cosine_similarity(dict_trs[i], dict_trs[j])[0])
        except:
            continue
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [25]:
cos_trans

,0,1,2,3,4,5,6,7,8,9,...,436,437,438,439,440,441,442,443,444,445
0,1.0,0.548764,0.550297,0.548661,0.547025,0.557532,0.548739,0.546148,0.548223,0.548875,...,0.646644,0.549346,0.553640,0.547782,0.548211,0.551163,0.548103,0.549924,0.549249,0.546359
1,NaN,1.000000,0.548756,0.548549,0.547539,0.543948,0.548187,0.545185,0.551053,0.544174,...,0.647635,0.550993,0.546912,0.550839,0.547065,0.548748,0.546057,0.549980,0.549696,0.548082
2,NaN,NaN,0.999999,0.548579,0.549407,0.549853,0.547376,0.548693,0.550044,0.547801,...,0.650223,0.551709,0.552520,0.548417,0.547838,0.549177,0.554755,0.553719,0.549611,0.551742
3,NaN,NaN,NaN,0.999999,0.547705,0.546601,0.547681,0.549513,0.550089,0.551030,...,0.649794,0.548824,0.546474,0.550876,0.553493,0.548565,0.557313,0.550542,0.549757,0.546190
4,NaN,NaN,NaN,NaN,0.999999,0.547328,0.547383,0.548014,0.549887,0.547764,...,0.648461,0.547935,0.547472,0.551091,0.552391,0.551499,0.546729,0.547708,0.547392,0.546201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.999999,0.551167,0.549376,0.552328,0.549701
442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.548953,0.550472,0.547200
443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.548999,0.549327
444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999999,0.549192


In [78]:
cmd.loc[1, :].min()

0.6535499095916748

# desc, translations 매트릭스화

In [27]:
for i in range(len(cmd)):
    for j in range(len(cmd)):
        if cmd.loc[i, j] >= 0:
            continue
        else:
            cmd.loc[i, j] = cmd.loc[j, i]

In [29]:
for i in range(len(cos_trans)):
    for j in range(len(cos_trans)):
        try:
            if cos_trans.loc[i, j]>0:
                continue
            else:
                cos_trans.loc[i, j] = cos_trans.loc[j, i]
        except:
            cos_trans.loc[i, j] = float(0)


In [49]:
cmd

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,1.000000,0.862349,0.686379,0.813836,0.855979,0.897495,0.805169,0.908984,0.925652,0.843775,...,0.882570,0.783158,0.892192,0.934559,0.934192,0.838997,0.924702,0.928860,0.928897,0.682974
1,0.862349,1.000000,0.658045,0.787599,0.831420,0.857652,0.777606,0.868193,0.884556,0.816533,...,0.849130,0.757342,0.854864,0.892339,0.892426,0.812372,0.883509,0.886782,0.887312,0.656235
2,0.686379,0.658045,0.999999,0.629073,0.654612,0.680691,0.622333,0.689552,0.706187,0.648605,...,0.670692,0.611793,0.677714,0.715021,0.714913,0.643452,0.706111,0.709497,0.709422,0.546697
3,0.813836,0.787599,0.629073,1.000000,0.784085,0.810384,0.748516,0.818251,0.835203,0.777100,...,0.800864,0.725389,0.806406,0.843753,0.843804,0.772075,0.833889,0.838294,0.838467,0.626168
4,0.855979,0.831420,0.654612,0.784085,1.000000,0.850644,0.775403,0.859348,0.877853,0.813254,...,0.842091,0.754702,0.849149,0.886576,0.886485,0.808703,0.877512,0.879575,0.880479,0.653365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,0.838997,0.812372,0.643452,0.772075,0.808703,0.833745,0.763556,0.843777,0.859304,0.800837,...,0.825737,0.741665,0.831959,0.868932,0.868111,0.999999,0.859699,0.862327,0.863097,0.640740
453,0.924702,0.883509,0.706111,0.833889,0.877512,0.917760,0.825189,0.932988,0.958032,0.864976,...,0.903318,0.803300,0.912832,0.967110,0.966773,0.859699,1.000000,0.960711,0.961763,0.703747
454,0.928860,0.886782,0.709497,0.838294,0.879575,0.919747,0.827803,0.935767,0.962177,0.868043,...,0.907247,0.807236,0.916347,0.972135,0.971639,0.862327,0.960711,1.000000,0.970280,0.706939
455,0.928897,0.887312,0.709422,0.838467,0.880479,0.920923,0.828489,0.936218,0.962278,0.867591,...,0.907286,0.807892,0.916547,0.973221,0.973382,0.863097,0.961763,0.970280,1.000000,0.706748


In [50]:
cmd.to_csv("./cos_sim_description.csv")

In [51]:
cos_trans.to_csv("./cos_translations.csv")